In [ ]:
import pennylane as qml
from pennylane import numpy as np
from qiskit import IBMQ
from pennylane import numpy as np

In [ ]:
#enable account
api_token = 'Your IBM Quantum Experience '
IBMQ.enable_account(api_token)
provider = IBMQ.get_provider(hub='ibm-q-hub-ntu', group='ntu-internal', project='default')
#provider = IBMQ.get_provider(hub='ibm-q', group='open', project='main')

In [ ]:
#看有哪些cloud backend可以使用，列出來
available_cloud_backends = provider.backends() 
print('Cloud backends:')
for i in available_cloud_backends: print(i)

In [ ]:
#遊戲地圖
from IPython.display import Image, display
display(Image(filename = "map.jpg", width = 600))

In [ ]:
#控制遊戲中上下走的unitary，有兩種
Unitary_Matrix_1 = np.array([[0,0,0,1],
                             [1,0,0,0],
                             [0,1,0,0],
                             [0,0,1,0]])

Unitary_Matrix_2 = np.array([[0,1,0,0],
                             [0,0,1,0],
                             [0,0,0,1],
                             [1,0,0,0]])


Go_Down = Unitary_Matrix_1
Go_Up = Unitary_Matrix_2
Go_Down_Dagger = Unitary_Matrix_2
Go_Up_Dagger = Unitary_Matrix_1

#PauliX的代表矩陣(qml.ControlledQubitUnitary()、qml.QubitUnitary()會用到)
paulix = np.array([[0,1],
                   [1,0]])
##PauliZ的代表矩陣(qml.ControlledQubitUnitary()、qml.QubitUnitary()會用到)
pauliz = np.array([[1,0],
                   [0,-1]])

In [ ]:
#用來將qml.probs()的output轉成較易讀的format
def transform_function(circuit_output,table_size):
    sortLoc = np.argsort(-1*circuit_output)
    sortLoc = np.expand_dims(sortLoc, axis = 0)
    sortVal = np.abs(np.sort(-1*circuit_output))
    sortVal = np.expand_dims(sortVal, axis = 0)
    Loc_Val_matrix = np.concatenate((sortLoc,sortVal),axis=0)
    print(f'The Quantum State Probability for greatest to least:\n')
    for i in range(np.shape(Loc_Val_matrix)[1]):
        if len(bin(int(Loc_Val_matrix[0,i].item()))[2:]) < table_size:
            num_of_zero =  table_size - len(bin(int(Loc_Val_matrix[0,i].item()))[2:])
            string = bin(int(Loc_Val_matrix[0,i].item()))[2:]
            for j in range(num_of_zero):
                string = '0' + string
            num_of_dash = 0
            for k in range(int(table_size/4)-1):
                string = string[:4 + (k * 4) + num_of_dash] + '-' + string[4 + (k * 4) + num_of_dash:table_size + num_of_dash]
                num_of_dash = num_of_dash + 1
            print(f'probability of {string} is {Loc_Val_matrix[1,i].item()}')
        else:
            string = bin(int(Loc_Val_matrix[0,i].item()))[2:]
            num_of_dash = 0
            for k in range(int(table_size/4)-1):
                string = string[:4 + (k * 4) + num_of_dash] + '-' + string[4 + (k * 4) + num_of_dash:table_size + num_of_dash]
                num_of_dash = num_of_dash + 1
            print(f'probability of {string} is {Loc_Val_matrix[1,i].item()}')

In [ ]:
#對quantum circuit的output做格式的轉換
def zero_one_measurement(x):
    y = np.where(x == 1,0,1)
    y = np.transpose(y)
    return y

In [ ]:
#算出二進位矩陣的十進位數字表示，會用在prepare_GA_quantum_state_vector()中
def bin_array_to_int(x):
    bin_str = "".join(list(map(str,x)))
    result = int('0b' + bin_str, 2)
    return result

#Input一個8 bit sequence，用來產生和其hamming distance差一的所有可能的8 bit sequences
#接著再用一個用均勻疊加態的quantum state來表示這所有可能的8 bit sequence
def prepare_GA_quantum_state_vector(target_sequence):
    #算出mutation_entry
    mutation_entry = np.zeros(len(target_sequence))
    for i in range(len(target_sequence)):
        if target_sequence[7-i] == 0:
            mutation_entry[i] = bin_array_to_int(target_sequence) + pow(2,i)
        elif target_sequence[7-i] == 1:
            mutation_entry[i] = bin_array_to_int(target_sequence) - pow(2,i)
    #算出quantum state vector
    quantum_state_vector = np.zeros(pow(2,len(target_sequence)))
    for i in range(len(target_sequence)):
        quantum_state_vector[int(mutation_entry[i])] = 1/np.sqrt(len(target_sequence))
    return quantum_state_vector

In [ ]:
#在抽出前兩個state的table的part若最後grover diffusion operator的部分沒有用到
#qml.MottonenStatePreparation(equally_superposition, wires=range(8))則不用執行這個block
dev = qml.device('default.qubit', wires=8)
@qml.qnode(dev)
def support_circuit2():
    for i in range(8):
        qml.Hadamard(i)
    return qml.state()
equally_superposition = support_circuit2()

### 先抽出前兩個steps的table

In [ ]:
#dev_previous = qml.device('qiskit.ibmq', wires = 15, backend = 'ibmq_guadalupe', provider = provider, shots = 1)
dev_previous2 = qml.device('default.qubit', wires = 15)
repeat_time_previous = 1

@qml.qnode(dev_previous2)
def circuit_previous():
    #使table為equally superposition的
    for i in range(8):
        qml.Hadamard(i)
    
    #為了做phase kickback的設置
    qml.PauliX(14)
    
    #設定初始位置（設為|01>）
    qml.PauliX(9)
    
    for k in range(repeat_time_previous):
        #將state 1的狀態copy到state 2
        qml.CNOT(wires=[8,10])
        qml.CNOT(wires=[9,11])
        
        #判斷在這個state 2要做哪一個action的control unitary(總共會有8個)    
        #在｜00> state所要做的action
        qml.QubitUnitary(qml.ControlledQubitUnitary.compute_matrix(Go_Down, control_wires = [0,8,9], u_wires = [10,11], control_values='000'), wires = [0,8,9,10,11])
        qml.QubitUnitary(qml.ControlledQubitUnitary.compute_matrix(Go_Up, control_wires = [0,8,9], u_wires = [10,11], control_values='100'), wires = [0,8,9,10,11])   
        #在｜01> state所要做的action
        qml.QubitUnitary(qml.ControlledQubitUnitary.compute_matrix(Go_Down, control_wires = [1,8,9], u_wires = [10,11], control_values='001'), wires = [1,8,9,10,11])
        qml.QubitUnitary(qml.ControlledQubitUnitary.compute_matrix(Go_Up, control_wires = [1,8,9], u_wires = [10,11], control_values='101'), wires = [1,8,9,10,11])   
        #在｜10> state所要做的action
        qml.QubitUnitary(qml.ControlledQubitUnitary.compute_matrix(Go_Down, control_wires = [2,8,9], u_wires = [10,11], control_values='010'), wires = [2,8,9,10,11])
        qml.QubitUnitary(qml.ControlledQubitUnitary.compute_matrix(Go_Up, control_wires = [2,8,9], u_wires = [10,11], control_values='110'), wires = [2,8,9,10,11])    
        #在｜11> state所要做的action
        qml.QubitUnitary(qml.ControlledQubitUnitary.compute_matrix(Go_Down, control_wires = [3,8,9], u_wires = [10,11], control_values='011'), wires = [3,8,9,10,11])
        qml.QubitUnitary(qml.ControlledQubitUnitary.compute_matrix(Go_Up, control_wires = [3,8,9], u_wires = [10,11], control_values='111'), wires = [3,8,9,10,11])
        
        #將state 2 copy到state 3
        qml.CNOT(wires=[10,12])
        qml.CNOT(wires=[11,13])
        
        #判斷在這個state 3要做哪一個action的control unitary
        #在｜00> state所要做的action
        qml.QubitUnitary(qml.ControlledQubitUnitary.compute_matrix(Go_Down, control_wires = [4,10,11], u_wires = [12,13], control_values='000'), wires = [4,10,11,12,13])
        qml.QubitUnitary(qml.ControlledQubitUnitary.compute_matrix(Go_Up, control_wires = [4,10,11], u_wires = [12,13], control_values='100'), wires = [4,10,11,12,13])   
        #在｜01> state所要做的action
        qml.QubitUnitary(qml.ControlledQubitUnitary.compute_matrix(Go_Down, control_wires = [5,10,11], u_wires = [12,13], control_values='001'), wires = [5,10,11,12,13])
        qml.QubitUnitary(qml.ControlledQubitUnitary.compute_matrix(Go_Up, control_wires = [5,10,11], u_wires = [12,13], control_values='101'), wires = [5,10,11,12,13])
        #在｜11> state所要做的action
        qml.QubitUnitary(qml.ControlledQubitUnitary.compute_matrix(Go_Down, control_wires = [7,10,11], u_wires = [12,13], control_values='011'), wires = [7,10,11,12,13])
        qml.QubitUnitary(qml.ControlledQubitUnitary.compute_matrix(Go_Up, control_wires = [7,10,11], u_wires = [12,13], control_values='111'), wires = [7,10,11,12,13])
        #將|10> state(洞)塞到|11> state (下一個round的洞)
        qml.QubitUnitary(qml.ControlledQubitUnitary.compute_matrix(Go_Down, control_wires = [10,11], u_wires = [12,13], control_values='10'), wires = [10,11,12,13])
        
        #判斷state 3沒有成功走在ok的路上(翻相位)
        qml.QubitUnitary(qml.ControlledQubitUnitary.compute_matrix(pauliz, control_wires = [12,13], u_wires = [14], control_values='00'), wires = [12,13,14])
        qml.QubitUnitary(qml.ControlledQubitUnitary.compute_matrix(pauliz, control_wires = [12,13], u_wires = [14], control_values='01'), wires = [12,13,14])
        qml.QubitUnitary(qml.ControlledQubitUnitary.compute_matrix(pauliz, control_wires = [12,13], u_wires = [14], control_values='10'), wires = [12,13,14])
        
        ######開始回復######
        
        #判斷在這個state 3要做哪一個action的control unitary
        #在｜00> state所要做的action
        qml.QubitUnitary(qml.ControlledQubitUnitary.compute_matrix(Go_Down_Dagger, control_wires = [4,10,11], u_wires = [12,13], control_values='000'), wires = [4,10,11,12,13])
        qml.QubitUnitary(qml.ControlledQubitUnitary.compute_matrix(Go_Up_Dagger, control_wires = [4,10,11], u_wires = [12,13], control_values='100'), wires = [4,10,11,12,13])   
        #在｜01> state所要做的action
        qml.QubitUnitary(qml.ControlledQubitUnitary.compute_matrix(Go_Down_Dagger, control_wires = [5,10,11], u_wires = [12,13], control_values='001'), wires = [5,10,11,12,13])
        qml.QubitUnitary(qml.ControlledQubitUnitary.compute_matrix(Go_Up_Dagger, control_wires = [5,10,11], u_wires = [12,13], control_values='101'), wires = [5,10,11,12,13])
        #在｜11> state所要做的action
        qml.QubitUnitary(qml.ControlledQubitUnitary.compute_matrix(Go_Down_Dagger, control_wires = [7,10,11], u_wires = [12,13], control_values='011'), wires = [7,10,11,12,13])
        qml.QubitUnitary(qml.ControlledQubitUnitary.compute_matrix(Go_Up_Dagger, control_wires = [7,10,11], u_wires = [12,13], control_values='111'), wires = [7,10,11,12,13])
        #將|10> state(洞)塞到|11> state (下一個round的洞)
        qml.QubitUnitary(qml.ControlledQubitUnitary.compute_matrix(Go_Down_Dagger, control_wires = [10,11], u_wires = [12,13], control_values='10'), wires = [10,11,12,13])
        
        #將state 2 copy到state 3
        qml.CNOT(wires=[10,12])
        qml.CNOT(wires=[11,13])
        
        #判斷在這個state 2要做哪一個action的control unitary(總共會有8個)    
        #在｜00> state所要做的action
        qml.QubitUnitary(qml.ControlledQubitUnitary.compute_matrix(Go_Down_Dagger, control_wires = [0,8,9], u_wires = [10,11], control_values='000'), wires = [0,8,9,10,11])
        qml.QubitUnitary(qml.ControlledQubitUnitary.compute_matrix(Go_Up_Dagger, control_wires = [0,8,9], u_wires = [10,11], control_values='100'), wires = [0,8,9,10,11])   
        #在｜01> state所要做的action
        qml.QubitUnitary(qml.ControlledQubitUnitary.compute_matrix(Go_Down_Dagger, control_wires = [1,8,9], u_wires = [10,11], control_values='001'), wires = [1,8,9,10,11])
        qml.QubitUnitary(qml.ControlledQubitUnitary.compute_matrix(Go_Up_Dagger, control_wires = [1,8,9], u_wires = [10,11], control_values='101'), wires = [1,8,9,10,11])   
        #在｜10> state所要做的action
        qml.QubitUnitary(qml.ControlledQubitUnitary.compute_matrix(Go_Down_Dagger, control_wires = [2,8,9], u_wires = [10,11], control_values='010'), wires = [2,8,9,10,11])
        qml.QubitUnitary(qml.ControlledQubitUnitary.compute_matrix(Go_Up_Dagger, control_wires = [2,8,9], u_wires = [10,11], control_values='110'), wires = [2,8,9,10,11])    
        #在｜11> state所要做的action
        qml.QubitUnitary(qml.ControlledQubitUnitary.compute_matrix(Go_Down_Dagger, control_wires = [3,8,9], u_wires = [10,11], control_values='011'), wires = [3,8,9,10,11])
        qml.QubitUnitary(qml.ControlledQubitUnitary.compute_matrix(Go_Up_Dagger, control_wires = [3,8,9], u_wires = [10,11], control_values='111'), wires = [3,8,9,10,11])
        
        #將state 1的狀態copy到state 2
        qml.CNOT(wires=[8,10])
        qml.CNOT(wires=[9,11])
        
        ########以下是grover diffusion operator########
        '''
        for j in range(8):
            qml.Hadamard(j)
            qml.PauliX(j)
        qml.QubitUnitary(qml.ControlledQubitUnitary.compute_matrix(pauliz, control_wires = [0,1,2,3,4,5,6], u_wires = [7], control_values='1111111'), wires = range(8))
        for j in range(8):
            qml.PauliX(j)
            qml.Hadamard(j)
        '''
        ########用qml.MottonenStatePreparation()#######
        #qml.MottonenStatePreparation(equally_superposition, wires=range(8)).inv()
        for i in range(8):
            qml.Hadamard(i)
        ###################
        for k in range(8):
            qml.PauliX(k)
        qml.Hadamard(7)
        qml.MultiControlledX(wires = [0,1,2,3,4,5,6,7], work_wires = [8])
        qml.Hadamard(7)
        for k in range(8):
            qml.PauliX(k)
        ###################         
        #qml.MottonenStatePreparation(equally_superposition, wires=range(8))
        for i in range(8):
            qml.Hadamard(i)
        #return qml.probs(wires=[0,1,2,3,4,5,6,7])
        #return qml.probs(wires=[8,9,10,11,12,13,14])
        return [qml.expval(qml.PauliZ(i)) for i in range(8)]

In [ ]:
#抽出來
#transform_function(circuit_previous(),8)
initial_table = initial_table.numpy().tolist()
initial_table

### 跑整個地圖

In [ ]:
#創建Environment的quantum circuit
#def Environment(Grover_diffusion_operator,initial_table):
#Problem:發現用qml.QubitUnitary()跑在real quantum device上容量瞬間爆掉，因為把qml.QubitUnitary()拿掉後可以順利跑
dev = qml.device('qiskit.ibmq', wires=27, backend = 'ibm_auckland', provider = provider)
dev2 = qml.device('default.qubit', wires=27)
repeat_time = 1
@qml.qnode(dev2)
def circuit_GA():
    #先prepare通過Mutation所得到的quantum state
    qml.QubitStateVector(prepare_GA_quantum_state_vector(initial_table), wires=range(8))
    #再來是equally superposition
    for i in range(8):
        qml.Hadamard(i+8)

    #為了做phase kickback的設置
    qml.PauliX(26)

    #設定初始位置（設為|01>）
    qml.PauliX(17)

    for k in range(repeat_time):
        #將state 1的狀態copy到state 2
        qml.CNOT(wires=[16,18])
        qml.CNOT(wires=[17,19])

        #判斷在這個state 2要做哪一個action的control unitary(總共會有8個)    
        #在｜00> state所要做的action
        qml.QubitUnitary(qml.ControlledQubitUnitary.compute_matrix(Go_Down, control_wires = [0,16,17], u_wires = [18,19], control_values='000'), wires = [0,16,17,18,19])
        qml.QubitUnitary(qml.ControlledQubitUnitary.compute_matrix(Go_Up, control_wires = [0,16,17], u_wires = [18,19], control_values='100'), wires = [0,16,17,18,19])   
        #在｜01> state所要做的action
        qml.QubitUnitary(qml.ControlledQubitUnitary.compute_matrix(Go_Down, control_wires = [1,16,17], u_wires = [18,19], control_values='001'), wires = [1,16,17,18,19])
        qml.QubitUnitary(qml.ControlledQubitUnitary.compute_matrix(Go_Up, control_wires = [1,16,17], u_wires = [18,19], control_values='101'), wires = [1,16,17,18,19])   
        #在｜10> state所要做的action
        qml.QubitUnitary(qml.ControlledQubitUnitary.compute_matrix(Go_Down, control_wires = [2,16,17], u_wires = [18,19], control_values='010'), wires = [2,16,17,18,19])
        qml.QubitUnitary(qml.ControlledQubitUnitary.compute_matrix(Go_Up, control_wires = [2,16,17], u_wires = [18,19], control_values='110'), wires = [2,16,17,18,19])    
        #在｜11> state所要做的action
        qml.QubitUnitary(qml.ControlledQubitUnitary.compute_matrix(Go_Down, control_wires = [3,16,17], u_wires = [18,19], control_values='011'), wires = [3,16,17,18,19])
        qml.QubitUnitary(qml.ControlledQubitUnitary.compute_matrix(Go_Up, control_wires = [3,16,17], u_wires = [18,19], control_values='111'), wires = [3,16,17,18,19])

        #將state 2 copy到state 3
        qml.CNOT(wires=[18,20])
        qml.CNOT(wires=[19,21])

        #判斷在這個state 3要做哪一個action的control unitary
        #在｜00> state所要做的action
        qml.QubitUnitary(qml.ControlledQubitUnitary.compute_matrix(Go_Down, control_wires = [4,18,19], u_wires = [20,21], control_values='000'), wires = [4,18,19,20,21])
        qml.QubitUnitary(qml.ControlledQubitUnitary.compute_matrix(Go_Up, control_wires = [4,18,19], u_wires = [20,21], control_values='100'), wires = [4,18,19,20,21])   
        #在｜01> state所要做的action
        qml.QubitUnitary(qml.ControlledQubitUnitary.compute_matrix(Go_Down, control_wires = [5,18,19], u_wires = [20,21], control_values='001'), wires = [5,18,19,20,21])
        qml.QubitUnitary(qml.ControlledQubitUnitary.compute_matrix(Go_Up, control_wires = [5,18,19], u_wires = [20,21], control_values='101'), wires = [5,18,19,20,21])
        #在｜11> state所要做的action
        qml.QubitUnitary(qml.ControlledQubitUnitary.compute_matrix(Go_Down, control_wires = [7,18,19], u_wires = [20,21], control_values='011'), wires = [7,18,19,20,21])
        qml.QubitUnitary(qml.ControlledQubitUnitary.compute_matrix(Go_Up, control_wires = [7,18,19], u_wires = [20,21], control_values='111'), wires = [7,18,19,20,21])
        #將|10> state(洞)塞到|11> state (下一個round的洞)
        qml.QubitUnitary(qml.ControlledQubitUnitary.compute_matrix(Go_Down, control_wires = [18,19], u_wires = [20,21], control_values='10'), wires = [18,19,20,21])

        #將state 3 copy到state 4
        qml.CNOT(wires=[20,22])
        qml.CNOT(wires=[21,23])

        #判斷在這個state 4要做哪一個action的control unitary
        #在｜00> state所要做的action
        qml.QubitUnitary(qml.ControlledQubitUnitary.compute_matrix(Go_Down, control_wires = [8,20,21], u_wires = [22,23], control_values='000'), wires = [8,20,21,22,23])
        qml.QubitUnitary(qml.ControlledQubitUnitary.compute_matrix(Go_Up, control_wires = [8,20,21], u_wires = [22,23], control_values='100'), wires = [8,20,21,22,23])   
        #在｜01> state所要做的action
        qml.QubitUnitary(qml.ControlledQubitUnitary.compute_matrix(Go_Down, control_wires = [9,20,21], u_wires = [22,23], control_values='001'), wires = [9,20,21,22,23])
        qml.QubitUnitary(qml.ControlledQubitUnitary.compute_matrix(Go_Up, control_wires = [9,20,21], u_wires = [22,23], control_values='101'), wires = [9,20,21,22,23])  
        #在｜10> state所要做的action
        qml.QubitUnitary(qml.ControlledQubitUnitary.compute_matrix(Go_Down, control_wires = [10,20,21], u_wires = [22,23], control_values='010'), wires = [10,20,21,22,23])
        qml.QubitUnitary(qml.ControlledQubitUnitary.compute_matrix(Go_Up, control_wires = [10,20,21], u_wires = [22,23], control_values='110'), wires = [10,20,21,22,23])  
        #將|11> state(洞)塞到|10> state (下一個round的洞)
        qml.QubitUnitary(qml.ControlledQubitUnitary.compute_matrix(Go_Up, control_wires = [20,21], u_wires = [22,23], control_values='11'), wires = [20,21,22,23])

        #將state 4 copy到state 5
        qml.CNOT(wires=[22,24])
        qml.CNOT(wires=[23,25])

        #判斷在這個state 5做哪一個action的control unitary
        #在｜00> state所要做的action
        qml.QubitUnitary(qml.ControlledQubitUnitary.compute_matrix(Go_Down, control_wires = [12,22,23], u_wires = [24,25], control_values='000'), wires = [12,22,23,24,25])  
        qml.QubitUnitary(qml.ControlledQubitUnitary.compute_matrix(Go_Up, control_wires = [12,22,23], u_wires = [24,25], control_values='100'), wires = [12,22,23,24,25])     
        #在｜01> state所要做的action
        qml.QubitUnitary(qml.ControlledQubitUnitary.compute_matrix(Go_Down, control_wires = [13,22,23], u_wires = [24,25], control_values='001'), wires = [13,22,23,24,25])  
        qml.QubitUnitary(qml.ControlledQubitUnitary.compute_matrix(Go_Up, control_wires = [13,22,23], u_wires = [24,25], control_values='101'), wires = [13,22,23,24,25])     
        #在｜11> state所要做的action
        qml.QubitUnitary(qml.ControlledQubitUnitary.compute_matrix(Go_Down, control_wires = [15,22,23], u_wires = [24,25], control_values='011'), wires = [15,22,23,24,25])  
        qml.QubitUnitary(qml.ControlledQubitUnitary.compute_matrix(Go_Up, control_wires = [15,22,23], u_wires = [24,25], control_values='111'), wires = [15,22,23,24,25])    

        #判斷state 5沒有成功走到終點!(翻相位)
        qml.QubitUnitary(qml.ControlledQubitUnitary.compute_matrix(pauliz, control_wires = [24,25], u_wires = [26], control_values='01'), wires = [24,25,26])


        ######開始回復######
         #判斷在這個state 5做哪一個action的control unitary
        #在｜00> state所要做的action
        qml.QubitUnitary(qml.ControlledQubitUnitary.compute_matrix(Go_Down_Dagger, control_wires = [12,22,23], u_wires = [24,25], control_values='000'), wires = [12,22,23,24,25])  
        qml.QubitUnitary(qml.ControlledQubitUnitary.compute_matrix(Go_Up_Dagger, control_wires = [12,22,23], u_wires = [24,25], control_values='100'), wires = [12,22,23,24,25])     
        #在｜01> state所要做的action
        qml.QubitUnitary(qml.ControlledQubitUnitary.compute_matrix(Go_Down_Dagger, control_wires = [13,22,23], u_wires = [24,25], control_values='001'), wires = [13,22,23,24,25])  
        qml.QubitUnitary(qml.ControlledQubitUnitary.compute_matrix(Go_Up_Dagger, control_wires = [13,22,23], u_wires = [24,25], control_values='101'), wires = [13,22,23,24,25])     
        #在｜11> state所要做的action
        qml.QubitUnitary(qml.ControlledQubitUnitary.compute_matrix(Go_Down_Dagger, control_wires = [15,22,23], u_wires = [24,25], control_values='011'), wires = [15,22,23,24,25])  
        qml.QubitUnitary(qml.ControlledQubitUnitary.compute_matrix(Go_Up_Dagger, control_wires = [15,22,23], u_wires = [24,25], control_values='111'), wires = [15,22,23,24,25]) 

        #將state 4 copy到state 5
        qml.CNOT(wires=[22,24])
        qml.CNOT(wires=[23,25])

        #判斷在這個state 4要做哪一個action的control unitary
        #在｜00> state所要做的action
        qml.QubitUnitary(qml.ControlledQubitUnitary.compute_matrix(Go_Down_Dagger, control_wires = [8,20,21], u_wires = [22,23], control_values='000'), wires = [8,20,21,22,23])
        qml.QubitUnitary(qml.ControlledQubitUnitary.compute_matrix(Go_Up_Dagger, control_wires = [8,20,21], u_wires = [22,23], control_values='100'), wires = [8,20,21,22,23])   
        #在｜01> state所要做的action
        qml.QubitUnitary(qml.ControlledQubitUnitary.compute_matrix(Go_Down_Dagger, control_wires = [9,20,21], u_wires = [22,23], control_values='001'), wires = [9,20,21,22,23])
        qml.QubitUnitary(qml.ControlledQubitUnitary.compute_matrix(Go_Up_Dagger, control_wires = [9,20,21], u_wires = [22,23], control_values='101'), wires = [9,20,21,22,23])  
        #在｜10> state所要做的action
        qml.QubitUnitary(qml.ControlledQubitUnitary.compute_matrix(Go_Down_Dagger, control_wires = [10,20,21], u_wires = [22,23], control_values='010'), wires = [10,20,21,22,23])
        qml.QubitUnitary(qml.ControlledQubitUnitary.compute_matrix(Go_Up_Dagger, control_wires = [10,20,21], u_wires = [22,23], control_values='110'), wires = [10,20,21,22,23])  
        #將|11> state(洞)塞到|10> state (下一個round的洞)
        qml.QubitUnitary(qml.ControlledQubitUnitary.compute_matrix(Go_Up_Dagger, control_wires = [20,21], u_wires = [22,23], control_values='11'), wires = [20,21,22,23])

        #將state 3 copy到state 4
        qml.CNOT(wires=[20,22])
        qml.CNOT(wires=[21,23])

        #判斷在這個state 3要做哪一個action的control unitary
        #在｜00> state所要做的action
        qml.QubitUnitary(qml.ControlledQubitUnitary.compute_matrix(Go_Down_Dagger, control_wires = [4,18,19], u_wires = [20,21], control_values='000'), wires = [4,18,19,20,21])
        qml.QubitUnitary(qml.ControlledQubitUnitary.compute_matrix(Go_Up_Dagger, control_wires = [4,18,19], u_wires = [20,21], control_values='100'), wires = [4,18,19,20,21])   
        #在｜01> state所要做的action
        qml.QubitUnitary(qml.ControlledQubitUnitary.compute_matrix(Go_Down_Dagger, control_wires = [5,18,19], u_wires = [20,21], control_values='001'), wires = [5,18,19,20,21])
        qml.QubitUnitary(qml.ControlledQubitUnitary.compute_matrix(Go_Up_Dagger, control_wires = [5,18,19], u_wires = [20,21], control_values='101'), wires = [5,18,19,20,21])
        #在｜11> state所要做的action
        qml.QubitUnitary(qml.ControlledQubitUnitary.compute_matrix(Go_Down_Dagger, control_wires = [7,18,19], u_wires = [20,21], control_values='011'), wires = [7,18,19,20,21])
        qml.QubitUnitary(qml.ControlledQubitUnitary.compute_matrix(Go_Up_Dagger, control_wires = [7,18,19], u_wires = [20,21], control_values='111'), wires = [7,18,19,20,21])
        #將|10> state(洞)塞到|11> state (下一個round的洞)
        qml.QubitUnitary(qml.ControlledQubitUnitary.compute_matrix(Go_Down_Dagger, control_wires = [18,19], u_wires = [20,21], control_values='10'), wires = [18,19,20,21])

        #將state 2 copy到state 3
        qml.CNOT(wires=[18,20])
        qml.CNOT(wires=[19,21])

        #判斷在這個state 2要做哪一個action的control unitary(總共會有8個)    
        #在｜00> state所要做的action
        qml.QubitUnitary(qml.ControlledQubitUnitary.compute_matrix(Go_Down_Dagger, control_wires = [0,16,17], u_wires = [18,19], control_values='000'), wires = [0,16,17,18,19])
        qml.QubitUnitary(qml.ControlledQubitUnitary.compute_matrix(Go_Up_Dagger, control_wires = [0,16,17], u_wires = [18,19], control_values='100'), wires = [0,16,17,18,19])   
        #在｜01> state所要做的action
        qml.QubitUnitary(qml.ControlledQubitUnitary.compute_matrix(Go_Down_Dagger, control_wires = [1,16,17], u_wires = [18,19], control_values='001'), wires = [1,16,17,18,19])
        qml.QubitUnitary(qml.ControlledQubitUnitary.compute_matrix(Go_Up_Dagger, control_wires = [1,16,17], u_wires = [18,19], control_values='101'), wires = [1,16,17,18,19])   
        #在｜10> state所要做的action
        qml.QubitUnitary(qml.ControlledQubitUnitary.compute_matrix(Go_Down_Dagger, control_wires = [2,16,17], u_wires = [18,19], control_values='010'), wires = [2,16,17,18,19])
        qml.QubitUnitary(qml.ControlledQubitUnitary.compute_matrix(Go_Up_Dagger, control_wires = [2,16,17], u_wires = [18,19], control_values='110'), wires = [2,16,17,18,19])    
        #在｜11> state所要做的action
        qml.QubitUnitary(qml.ControlledQubitUnitary.compute_matrix(Go_Down_Dagger, control_wires = [3,16,17], u_wires = [18,19], control_values='011'), wires = [3,16,17,18,19])
        qml.QubitUnitary(qml.ControlledQubitUnitary.compute_matrix(Go_Up_Dagger, control_wires = [3,16,17], u_wires = [18,19], control_values='111'), wires = [3,16,17,18,19])

        #將state 1的狀態copy到state 2
        qml.CNOT(wires=[16,18])
        qml.CNOT(wires=[17,19])

        ########以下是grover diffusion operator########
        ##有包含前一個step的table的grover operator
        qml.MottonenStatePreparation(prepare_GA_quantum_state_vector(initial_table), wires=range(8)).inv()
        for j in range(8):
            qml.Hadamard(j+8)
        ###################
        for k in range(16):
            qml.PauliX(k)
        qml.Hadamard(15)
        qml.MultiControlledX(wires = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15], work_wires = [16])
        qml.Hadamard(15)
        for k in range(16):
            qml.PauliX(k)
        ###################
        for j in range(8):
            qml.Hadamard(j+8)         
        qml.MottonenStatePreparation(prepare_GA_quantum_state_vector(initial_table), wires=range(8))
    return qml.probs(wires=[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15])
        #return [qml.expval(qml.PauliZ(i)) for i in range(16)]
#return circuit(initial_table)

In [ ]:
#zero_one_measurement(circuit_GA())
transform_function(circuit_GA(),16)